In [ ]:
##here we are plotting the exponential fit and Bolztmann fit of the pp product, Pb Pb data 

In [2]:
import ROOT as r
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
PbPb_pp_file = r.TFile("ROOTfile/PbPb_pp5tev.root")
PbPb_pp_file

In [3]:
Histo = PbPb_pp_file.Get("Table 5")
#Histo.ls()

In [4]:
#The first histogram corresponds to very low centrality (0-5%)

Y1 = Histo.Get("Hist1D_y1;1")
e1 = Histo.Get("Hist1D_y1_e1")
e2 = Histo.Get("Hist1D_y1_e2")
e3 = Histo.Get("Hist1D_y1_e3")


print(e2.GetBinError(20))
#e1.ls()

# This part is to avoid the kernel to crash, so I fix the memory position of my base plot
y1_plot  = Y1.Clone("y1_plot")
y1_plot.SetDirectory(0)


1.3297


In [5]:
#Here I calculated the squared sum of uncertainties and place it for each bin

for i in range(0, y1_plot.GetNbinsX()): #nb of bins
    err = math.sqrt( e1.GetBinContent(i)**2 + e2.GetBinContent(i)**2 +  e3.GetBinContent(i)**2)
    y1_plot.SetBinError(i, err) 

In [6]:
f = r.TF1("exp", "[0]*x/([1]*([2]+[1]))*exp(-sqrt(x*x+[2]*[2])/[1])", 0, 20)

In [7]:
f.SetParName(0, "A")
f.SetParName(1, "T")
f.SetParName(2, "mp")

f.SetParameter(0, 1)
f.SetParameter(1, 1)
f.SetParameter(2, 0.938272)
f.FixParameter(2, 0.938272)

In [8]:
y1_plot.Fit(f,"RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1425.04
NDf                       =           49
Edm                       =  4.34469e-08
NCalls                    =          170
A                         =      390.857   +/-   5.01567     
T                         =     0.550793   +/-   0.00162251  
mp                        =     0.938272                      	 (fixed)


In [9]:
c = r.TCanvas("c", "Data + exponential Fit")

y1_plot.SetTitle("pt distribution of p+p- in PbPb collision at 5 Tev")
y1_plot.GetXaxis().SetTitle("pt [Gev/c]")
y1_plot.GetYaxis().SetTitle("(1/Nev)*D²(N)/DptDyrap [(Gev/c)^⁻1]")

y1_plot.Draw("")
c.SetLogy()

f.SetLineColor(r.kRed)
f.Draw("SAME")

c.SaveAs("y1_fit_exp.pdf")

Info in <TCanvas::Print>: pdf file y1_fit_exp.pdf has been created


In [10]:
##programm : imrpove performance of the fit, change the position of the bins to make it corresponds,register the fit in a differents file each time we do it

In [11]:
f.ls()

OBJ: TF1	exp	[0]*x/([1]*([2]+[1]))*exp(-sqrt(x*x+[2]*[2])/[1]) : 0 at: 0x5d8e824e6020


In [12]:
print(f.Integral(0,0.05))
print(f.Integral(0.05,0.10))




0.10831368150741148
0.32337724274752694


In [13]:
##We want to compare the data and the fit, and it requires to plot the data on the x position of the mean of the fit function in each bin, this is the goal of this function    

In [14]:
def reposition_data(fitfunction, data):
    
    a=0
    list_newx = np.zeros(data.GetNbinsX()+1)
    
    for i in range(0, data.GetNbinsX()+1):

        width = data.GetBinWidth(i)
        
        a = data.GetBinLowEdge(i)
        b = a + width
    
        integral = fitfunction.Integral(a,b)
        mean = integral/width
        
        new_x = f.GetX(mean, a, b)

        list_newx[i] = new_x

    return list_newx


In [15]:
y1_plot.GetXaxis()

In [16]:
xlist = reposition_data(f, y1_plot)
print(xlist, len(xlist))

[ 0.10309911  0.324793    0.37475506  0.42471269  0.47466355  0.52460372
  0.57452631  0.62441774  0.67424727  0.72392701  0.77306732  0.81526291
  0.87719131  0.92598049  0.97559523  1.05136606  1.15074202  1.25040531
  1.35019253  1.450045    1.54993634  1.64985284  1.74978662  1.84973283
  1.94968828  2.09853915  2.29831779  2.49814893  2.69801619  2.89790933
  3.09782161  3.29774841  3.49768647  3.69763344  3.89758756  4.2345777
  4.73411694  5.23376513  5.73348816  6.23326469  6.73308073  7.4326034
  8.4317326   9.43107252 10.43055528 11.43013921 12.42979739 13.42951162
 14.4292692  15.42906098 16.73328681 18.73233755] 52


In [17]:
#We want to plot each point at its respective new x value 

g = r.TGraphErrors(y1_plot.GetNbinsX())

for i in range (1,y1_plot.GetNbinsX()+1):
    
    y  = y1_plot.GetBinContent(i)
    x  = xlist[i]
    ey = y1_plot.GetBinError(i)
    
    g.SetPoint(i, x, y)
    g.SetPointError(i, 0, ey)
    

print(ey,y,x)
print(y1_plot.GetNbinsX())
g.ls()

5.598e-05 5.59799991606269e-05 18.73233754919862
51
OBJ: TGraphErrors	Graph	Graph : 0 at: 0x5d8e83a6d060


In [ ]:
c2 = r.TCanvas("c2", "Data + exponential Fit with modified x")
c2.SetGrid()

g.SetTitle("pt distribution of p+p- in PbPb collision at 5 Tev")
g.GetXaxis().SetTitle("pt [Gev/c]")
g.GetYaxis().SetTitle("(1/Nev)*D²(N)/DptDyrap [(Gev/c)^⁻1]")

g.Draw("AP") 

f.SetLineColor(r.kRed)
f.Draw("SAME")

legend = r.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(g, "pp data", "lep")
legend.AddEntry(f, "exponential fit")
legend.Draw()

CHI = str(f.GetChisquare())[:8]
NDF = str(f.GetNDF())

Text = r.TPaveText(6,10,12,15)
Text.AddText(f" chi^2 = {CHI}")
Text.AddText(f" NdF = {NDF} ")
Text.Draw()

c2.Draw()
#c2.SaveAs("y1_fit_exp_GOODX.pdf")


##Add of information about the goodness of the fit

In [ ]:
f.GetChisquare()
f.GetParameter(0) #parameter nb 0 is A, parameter nb 1 is T

In [21]:
print(str(f.GetChisquare())[:8])


1425.036


In [3]:
PbPb_phiphi_file = r.TFile("ROOTfile/PbPb_PhiPhi5tev.root")
PbPb_phiphi_file.ls()

TFile**		ROOTfile/PbPb_PhiPhi5tev.root	
 TFile*		ROOTfile/PbPb_PhiPhi5tev.root	
  KEY: TDirectoryFile	Table 3;1	Table 3


In [5]:
Histo_PhiPhi = PbPb_phiphi_file.Get("Table 3")
Histo_PhiPhi.ls()

TDirectoryFile*		Table 3	Table 3
 KEY: TH1F	Hist1D_y1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y1_e1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y1_e2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TGraphAsymmErrors	Graph1D_y1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y2_e1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y2_e2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TGraphAsymmErrors	Graph1D_y2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y3;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y3_e1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y3_e2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TGraphAsymmErrors	Graph1D_y3;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y4;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y4_e1;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TH1F	Hist1D_y4_e2;1	doi:10.17182/hepdata.93150.v1/t3
 KEY: TGraphAsymmErrors	Graph1D_y4;1	doi:10.17182/hepdata.931

In [ ]:


Histo_PhiPhi.Get("
err1 = Hist.Get(f"Hist1D_y{centrality}_e1")
err2 = Hist.Get(f"Hist1D_y{centrality}_e2")

    

    